# 第四章 非连续内存分配

## 4.1 分段机制

**为什么要进行非连续内存分配？**

连续内存分配的缺点：
* 分配给一个程序的物理内存是连续的，导致内存利用率低。
* 有外碎片和内碎片的问题。

非连续分配的优点：
* 一个程序的物理地址空间可以使非连续的。
* 更好的内存利用和管理。
* 允许共享代码与数据。
* 支持动态加载和动态链接。

非连续分配的缺点：需要建立虚拟地址和物理地址之间转换的映射。

**分段机制**：更好的分离和共享内存。

分段寻址方案中逻辑地址由一个二元组表示，段号（s）和段内偏移（addr），它有两种实现方式，段寄存器+地址寄存器方式和单地址方式。

![](./images/figure04-01.png)

段表由操作系统设置，每个段表项包含两个信息，段的起始位置和段的长度限制。

## 4.2 分页机制

分页机制与分段机制最大的区别在于，分页机制中的每一页大小不变。

* 物理页：划分物理内存至固定大小的帧（Frame）。
* 逻辑页：划分逻辑地址空间至固定大小的页（Page）。
* 建立方案：转换逻辑地址为物理地址（Page to Frame），使用页表和MMU/TLB实现。

**帧**：物理内存被分割为大小相等的帧，一个物理地址表示为一个二元组（f，o）。

* f - 帧号（F位，共有$2^F$个帧）。
* o - 帧内偏移（S位，每一个帧有$2^S$个字节）。
* 物理地址 = $2^S \times f + o$。

**页**：一个程序的逻辑地址空间被划分为大小相等的页，一个逻辑地址也表示为一个二元组（p，o）。

* p - 页号（P位，共有$2^P$个页）。
* o - 页内偏移（S位，每一个页有$2^S$个字节）。
* 逻辑地址 = $2^S \times p + o$。

> * 页内偏移的大小 = 帧内偏移的大小
> * 页号大小 <> 帧号大小

![](./images/figure04-02.png)

页表保存了逻辑地址-物理地址之间的映射关系。

## 4.3 页表

每个运行的程序都有一个页表，它属于程序的运行状态，会动态变化，由页表基地址寄存器（PTBR）指向页表的起始位置。

**页表项的内容**：

* 标志位（Flags），标识页表是否合法等信息。
* 帧号（f）。

**页表的问题**：

* 时间开销：访问一个内存单元需要2次内存访问，一次用于获取页表项，一次用于访问数据。
* 容量问题：页表可能非常大。

**时间开销解决方案**：缓存（caching）。

Translation Look-aside Buffer（TLB）位于CPU中，用于缓存近期访问过的页帧转换表项：
* 使用 associative memory（关联内存）实现，具备快速访问性能。
* 如果命中，物理页号可以很快被获取。
* 如果未命中，对应表象被更新到TLB中。
* 缓存的实现是软件实现还是硬件实现取决于CPU的特性。

## 4.4 多级页表

通过多级页表的方式，间接访问物理内存，解决页表过大的问题。

**二级页表**

![](./images/figure04-03.png)

**多级页表**：建立页表树结构。

## 4.5 反向页表

当有大地址空间（64-bits）时，前向映射表会变得繁琐。反向页表不是让页表与逻辑空间的大小相对应，而是让页表与物理地址空间的大小相对应。因为逻辑地址空间增长速度快于物理地址空间的。

**基于页寄存器（Page Registers）的方案**：Page Registers保存的是帧号-页号之间的映射关系，它的缺点在于查找开销大，而优点在于帧的范围与物理地址空间大小相关，所以Page Registers的大小固定。

**基于关联内存（Associative Memory）的方案**：如果帧数较少，页寄存器可以被放置在关联内存中，它的限制因素在于大量的关联内存非常昂贵、难以在单个时钟周期内完成、耗电。

**基于哈希（Hash）计算查找的方案**：$h(PID,p) \rightarrow f$，它的问题在于哈希函数会产生碰撞，并且内存开销问题依旧存在。
